<a href="https://colab.research.google.com/github/TraeDWill/Dataeng_class/blob/main/6_DataIntegration/TraeDataIntegration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Modules**

In [1]:
import pandas as pd
from google.colab import drive
import os

**Downloading the CSVs**

In [2]:
%%capture
! gdown 18UD2EGKwq3YjVMBPsS8kVz0e671p7vzV
! gdown 1xAWZ_tEjEAFsosCdax78lrWkLnUgoWSn

import pandas as pd

covid_df = pd.read_csv("/content/COVID_county_data.csv")
census_df = pd.read_csv("/content/acs2017_census_tract_data.csv")

**Aggregate The Stuff**

In [3]:
income = ['Income', 'IncomeErr', 'IncomePerCap', 'IncomePerCapError', 'Poverty']

def custom_agg(column):
    if column.name in income:
        return round(column.mean(), 2)
    elif column.dtype == 'int64' or column.dtype == 'float64':  # Check if the column type is int or float
        return column.sum()
    else:
        return column.iloc[0] # Keep non-numeric columns unchanged

county_df = census_df.groupby('County').agg(custom_agg)

**Drop useless and add ID**

In [4]:
county_df = county_df.drop("TractId", axis=1)
county_df['County_ID'] = range(1, len(county_df) + 1)

In [ ]:
def particulars(county):
  county_df.loc[county]
  population = county_df.loc[county]['TotalPop']
  pov_perc = county_df.loc[county]['Poverty']
  pci = county_df.loc[county]['IncomePerCap']
  print(f'''Population: {population}, Poverty Percentage: {pov_perc},
  Per Capita Income: {pci}''')
  return [population, pov_perc, pci]


In [ ]:
particulars('Malheur County')

**Maximum and Minimum along with Row ID**

In [ ]:
max = county_df['TotalPop'].max()
county_max = county_df['TotalPop'].idxmax()
min = county_df['TotalPop'].min()
county_min = county_df['TotalPop'].idxmin()

print(f'''County with Max Population: {county_max}, Population: {max}.
      County with Min Population: {county_min}, Population: {min}''')

**Covid Integration**

In [26]:
covid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050258 entries, 0 to 1050257
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype    
---  ------  --------------    -----    
 0   date    1050258 non-null  period[M]
 1   county  1050258 non-null  object   
 2   state   1050258 non-null  object   
 3   fips    1040589 non-null  float64  
 4   cases   1050258 non-null  int64    
 5   deaths  1027519 non-null  float64  
dtypes: float64(2), int64(1), object(2), period[M](1)
memory usage: 48.1+ MB


In [5]:
# Convert 'date' column to datetime
covid_df['date'] = pd.to_datetime(covid_df['date'])

# Convert 'date' column to month periods
covid_df['date'] = covid_df['date'].dt.to_period('M')

# Define custom aggregation function
def custom_agg(column):
    if column.dtype == 'int64' or column.dtype == 'float64':
        # Sum numeric columns
        return column.sum()
    else:
        # Keep non-numeric columns unchanged, take the first value
        return column.iloc[0]

# Group by 'county' and 'date' (month) and apply the custom aggregation function
grouped_df = covid_df.groupby(['county', 'date']).agg(custom_agg)

print(grouped_df)


                            state       fips  cases  deaths
county    date                                             
Abbeville 2020-03  South Carolina   585013.0     32     0.0
          2020-04  South Carolina  1350030.0    430     0.0
          2020-05  South Carolina  1395031.0   1080     0.0
          2020-06  South Carolina  1350030.0   2187     0.0
          2020-07  South Carolina  1395031.0   5959    65.0
...                           ...        ...    ...     ...
Ziebach   2020-10    South Dakota  1430247.0   2418    36.0
          2020-11    South Dakota  1384110.0   4405   115.0
          2020-12    South Dakota  1430247.0   8247   219.0
          2021-01    South Dakota  1430247.0  10011   256.0
          2021-02    South Dakota   922740.0   6715   180.0

[22193 rows x 4 columns]


In [ ]:
oregon_df = grouped_df[grouped_df['state'] == 'Oregon']
total_cases = oregon_df['cases'].sum()
total_deaths = oregon_df['deaths'].sum()

print(f"Total cases in Oregon: {total_cases}")
print(f"Total deaths in Oregon: {total_deaths}")

In [ ]:
def findstuff(df, county):
  county_data = df.loc[county]
  case1 = county_data.loc['2020-07']['cases']
  case1d = county_data.loc['2020-07']['deaths']
  case2 = county_data.loc['2021-01']['cases']
  case2d = county_data.loc['2021-01']['deaths']
  case3 = county_data.loc['2021-02']['cases']
  case3d = county_data.loc['2021-02']['deaths']
  print(f"""Cases and Deaths in {county}:\n
          September 2020: Cases: {case1} Deaths: {case1d} \n
          January 2021: Cases: {case2} Deaths: {case2d} \n
          February 2021: Cases: {case3} Deaths: {case3d}""")

  return county_data

MEGAZORDS COMBINE

In [ ]:
new_covid_df = grouped_df
new_census_df = county_df
new_covid_df.info()
new_census_df.info()

In [36]:
#Fixin stuff

new_covid_df_reset = new_covid_df.reset_index()
new_census_df_reset = new_census_df.reset_index().rename(columns={'index': 'county'})
new_census_df_reset['County'] = new_census_df_reset['County'].str.replace(' County', '')
new_census_df_reset = new_census_df_reset.rename(columns={'County': 'county'})

#Mergin stuff
merged_df = pd.merge(new_covid_df_reset, new_census_df_reset, on='county', how='left')

#Choppin stuff
def custom_agg(group):
    return pd.Series({
        'County_ID': group['County_ID'].iloc[0],  # Keep the first ID
        'TotalPop': group['TotalPop'].sum(),  # Sum of population
        'Poverty': group['Poverty'].mean(),  # Average poverty
        'PerCapitaIncome': group['IncomePerCap'].mean(),  # Average per capita income
        'cases': group['cases'].iloc[0],  # First value of cases
        'deaths': group['deaths'].iloc[0],  # First value of deaths
        'TotalCasesPer100k': round(group['cases'].iloc[0] / (group['TotalPop'].sum() / 100000),2),  # Calculate cases per 100k
        'TotalDeathPer100k': round(group['deaths'].iloc[0] / (group['TotalPop'].sum() / 100000),2)  # Calculate deaths per 100k
    })
result_df = merged_df.groupby('county').apply(custom_agg).reset_index()
print(result_df)

<ipython-input-36-010d3fdc5f6e>:20: RuntimeWarning: divide by zero encountered in scalar divide
  'TotalCasesPer100k': round(group['cases'].iloc[0] / (group['TotalPop'].sum() / 100000),2),  # Calculate cases per 100k
<ipython-input-36-010d3fdc5f6e>:21: RuntimeWarning: divide by zero encountered in scalar divide
  'TotalDeathPer100k': round(group['deaths'].iloc[0] / (group['TotalPop'].sum() / 100000),2)  # Calculate deaths per 100k
<ipython-input-36-010d3fdc5f6e>:21: RuntimeWarning: invalid value encountered in scalar divide
  'TotalDeathPer100k': round(group['deaths'].iloc[0] / (group['TotalPop'].sum() / 100000),2)  # Calculate deaths per 100k


                         county  County_ID   TotalPop  Poverty  \
0                     Abbeville        1.0   297456.0    22.18   
1                        Acadia        NaN        0.0      NaN   
2                      Accomack        3.0   394080.0    20.11   
3                           Ada        4.0  5221404.0    12.61   
4                         Adair        5.0   888828.0    25.91   
...                         ...        ...        ...      ...   
1925  Yukon-Koyukuk Census Area     1951.0    59983.0    25.75   
1926                       Yuma     1952.0  2572680.0    19.17   
1927                     Zapata     1953.0   158565.0    32.57   
1928                     Zavala     1954.0   133672.0    41.38   
1929                    Ziebach     1955.0    28370.0    43.10   

      PerCapitaIncome  cases  deaths  TotalCasesPer100k  TotalDeathPer100k  
0            19402.83   32.0     0.0              10.76               0.00  
1                 NaN   85.0     5.0                i

In [39]:
def info(county, df):
  result = df[df['county'] == county]
  printer = result[['Poverty','TotalCasesPer100k']]
  print(printer)

In [43]:
info('Harlan', result_df)

     Poverty  TotalCasesPer100k
756    31.62               8.06
